In [25]:
#Declaration of Required Libraries
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay


In [26]:
!pip install nltk

In [27]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [28]:
#Connecting datas for training and testing machine
data = pd.read_csv('twitter_training.csv',header=None)
v_data = pd.read_csv('twitter_validation.csv',header=None)

In [29]:
#open file twitter_training.csv
data

,0,1,2,3
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


In [30]:
#open file twitter_validation.csv
v_data

,0,1,2,3
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...
...,...,...,...,...
995,4891,GrandTheftAuto(GTA),Irrelevant,⭐️ Toronto is the arts and culture capital of ...
996,4359,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
997,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
998,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [31]:
# columns names
data.columns = ['#', 'refers to', 'sentiment', 'text']
v_data.columns = ['#', 'refers to', 'sentiment', 'text']


In [32]:
data.dropna(inplace=True, axis=0)

In [33]:
#elimination of duplicate data
data=data.drop_duplicates()
v_data=v_data.drop_duplicates()

In [34]:
#Enter tokens
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+https\S+", '',text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#','',text)
    text = re.sub(r'[^\w\s]','',text)
    text_tokens = word_tokenize(text)
    return " ".join(text_tokens)

In [35]:
#using traning data
data.text = data['text'].apply(data_processing)

<ipython-input-35-34bec8b8c428>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.text = data['text'].apply(data_processing)


In [36]:
##using test/validation dara
v_data.text = v_data['text'].apply(data_processing)

In [37]:
#make a list of words using Porter's algorithm
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data]
    return data

In [38]:
#Application of Porter's algorithm for each input data
data['text'] = data['text'].apply(lambda x: stemming(x))
v_data['text'] = v_data['text'].apply(lambda x: stemming(x))

<ipython-input-38-e11baf3aeb32>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: stemming(x))


In [39]:
#data vector extraction
vect = CountVectorizer(ngram_range=(1,2)).fit(data['text'])

In [41]:
#Applying vector search on feature names
feature_names = vect.get_feature_names_out()

In [42]:
#Application/transformation of vector markings to Porter's algorithm
x_train =  vect.transform(data['text'])
y_train = data['sentiment']
x_test =  vect.transform(v_data['text'])
y_test = v_data['sentiment']

In [43]:
#Print vector's data
print("Size of x_train:", (x_train.shape))
print("Size of y_train:", (y_train.shape))
print("Size of x_test:", (x_test.shape))
print("Size of y_test:", (y_test.shape))

Size of x_train: (71656, 359410)
Size of y_train: (71656,)
Size of x_test: (1000, 359410)
Size of y_test: (1000,)


In [44]:
#Applying predicting the probability of a categorically dependent variable and getting test accuracy
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
logreg_pred = logreg.predict(x_test)
logreg_acc = accuracy_score(logreg_pred, y_test)
print("test accuracy: {:.2f}%".format(logreg_acc*100))

test accuracy: 98.50%


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [45]:
#Derivation of the matrix calculated by Porter's algorithm with regression dependence and classification report
print(confusion_matrix(y_test, logreg_pred))
print("\n")
print(classification_report(y_test, logreg_pred))

[[167   2   1   2]
 [  0 263   2   1]
 [  0   0 281   4]
 [  1   1   1 274]]


              precision    recall  f1-score   support

  Irrelevant       0.99      0.97      0.98       172
    Negative       0.99      0.99      0.99       266
     Neutral       0.99      0.99      0.99       285
    Positive       0.98      0.99      0.98       277

    accuracy                           0.98      1000
   macro avg       0.99      0.98      0.98      1000
weighted avg       0.99      0.98      0.99      1000



In [46]:
#Using the Linear Support Vector Classifier
SVCmodel = LinearSVC()
SVCmodel.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [47]:
#Getting and outputting test accuracy
svc_pred = SVCmodel.predict(x_test)
svc_acc = accuracy_score(svc_pred, y_test)
print("test accuracy: {:.2f}%".format(svc_acc*100))

test accuracy: 99.10%


In [48]:
#Porter matrix output with regression and classification report using a linear support vector classifier
print(confusion_matrix(y_test, svc_pred))
print("\n")
print(classification_report(y_test, svc_pred))

[[171   0   0   1]
 [  0 265   0   1]
 [  0   0 281   4]
 [  1   0   2 274]]


              precision    recall  f1-score   support

  Irrelevant       0.99      0.99      0.99       172
    Negative       1.00      1.00      1.00       266
     Neutral       0.99      0.99      0.99       285
    Positive       0.98      0.99      0.98       277

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000

